In [67]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import json
import re
import numpy as np
import openai


from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
from pathlib import Path

from langchain.embeddings import OpenAIEmbeddings

### Loading datasets

In [22]:
dataset_wiki = load_dataset("jerteh/SrpWiki")

In [23]:
dataset_wiki

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3818535
    })
})

In [65]:
text_wiki = dataset_wiki['train']['text']
print(text_wiki[:10])
print(len(text_wiki))
print(type(text_wiki))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.', 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.', 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.', 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).', 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.', '1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere.', 'Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune.', 'Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje 

In [26]:
dataset_news = load_dataset("jerteh/SrpKorNews")

In [27]:
dataset_news

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 337
    })
})

In [28]:
text_news = dataset_news['train']['text']
print(len(text_news))
print(type(text_news))

337
<class 'list'>


In [29]:
dataset_sci = load_dataset("procesaur/STARS")

In [30]:
dataset_sci

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7559526
    })
})

In [31]:
text_sci = dataset_sci['train']['text']

In [32]:
print(text_sci[0])
print(len(text_sci))
print(type)

<s>Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 Uticaj brzine vazduha na aerosole formirane od 10 % emulzija 128 Strujanje aerosola vode u cevi</s>
7559526
<class 'list'>


In [37]:
news_mini = text_news[:1]
sci_mini = text_sci[:30]

### Creating sentences

In [58]:
def create_sentnecs(texts):
    sentences = []
    for text in texts:
        matches = re.findall(r'<s>(.*?)</s>', text)
        sentences.extend(matches)
    
    
    return sentences

In [59]:
# crate one function with create_sentences and flag for loop if dataset isWiki? 
def return_dic(sentences):
    sent_lst_dic = [{'sentence': x, 'id' : i} for i, x in enumerate(sentences)]
    return sent_lst_dic

In [63]:
sent_wiki = return_dic(text_wiki[:30])
pprint(sent_wiki[:3])
print(len(sent_wiki))


[{'id': 0,
  'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski '
              'latiničnog teksta.'},
 {'id': 1,
  'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u '
              'Microsoft Word-u, Netscape Composer-u i tekstualnom editoru '
              'Kate.'},
 {'id': 2,
  'sentence': 'U tekstu su takođe preporučeni standardni Unicode fontovi koji '
              'omogućavaju laku prenosivost teksta sa računara na računar ili '
              'za objavljivanje teksta na Internet.'}]
30


In [64]:
sent_news = create_sentnecs(news_mini)
sent_news = return_dic(sent_news[:100])
pprint(sent_news[:3])
print(len(sent_news))

[{'id': 0, 'sentence': 'Mečka na usijanom limenom krovu'},
 {'id': 1,
  'sentence': 'Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil '
              'Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 '
              'automobila, čija je ukupna cena premašila fantastičnih 550 '
              'hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta '
              'kola nose ime njegove ćerke.'},
 {'id': 2,
  'sentence': 'Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a '
              'istorija automobilizma je imala sreću da je Jelinekovoj '
              'mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - '
              'Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, '
              'prijatnost.'}]
100


In [61]:
sent_sci = create_sentnecs(sci_mini)
sent_sci = return_dic(sent_sci)
pprint(sent_sci[:3])
print(len(sent_sci))

[{'id': 0,
  'sentence': 'Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 '
              'Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 '
              'Uticaj brzine vazduha na aerosole formirane od 10 % emulzija '
              '128 Strujanje aerosola vode u cevi'},
 {'id': 1,
  'sentence': 'Završna razmatranja strujanja aerosola u cevi formiranog '
              'emulzije EO3'},
 {'id': 2,
  'sentence': 'Moguće strukture aerosol čestice NaCl: a) neporozna čvrsta, b) '
              'porozna čvrsta, c) čvrsta sa otvorenim džepovima rastovrenog '
              'NaCl, d) čvrsta čaura sa vodenim jezgrom, e) čvrsta NaCl '
              'čestica sa jezgrom vodenog rastvora NaCl, f) vodena kapljica '
              'NaCl.'}]
81


### Creating combined sentences

Source: https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/8a30b5710b3dd99ef2239fb60c7b54bc38d3613d/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

In [66]:
openai.api_key = os.getenv("OPENAI_API_KEY")
oaiembeds = OpenAIEmbeddings()

NameError: name 'openai' is not defined

In [ ]:
buffer_size = 1

In [ ]:
# can this be improved?
def combine_sentences(sentences, buffer_size=buffer_size):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

In [ ]:
def generate_embeddings(sentences):
    embeddings = oaiembeds.embed_documents([x['combined_sentence'] for x in sentences])
    for i, sentence in enumerate(sentences):
        sentence['combined_sentence_embedding'] = embeddings[i]
    
    return sentences

In [ ]:
# why convert cosine similarity to cosine distance?
# just use cosine similarity instead?
# test on couple of examples and see if it makes a difference
def calculate_cosine_distances(sentences):
    distances = []
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        
        # Calculate cosine similarity
        similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
        
        # Convert to cosine distance
        distance = 1 - similarity

        # Append cosine distance to the list
        distances.append(distance)

        # Store distance in the dictionary
        sentences[i]['distance_to_next'] = distance

    # Optionally handle the last sentence
    # sentences[-1]['distance_to_next'] = None  # or a default value

    return distances, sentences

In [ ]:
def get_breakpoint(sentences, distances, treshold):
    # We need to get the distance threshold that we'll consider an outlier
    # We'll use numpy .percentile() for this
    breakpoint_percentile_threshold = treshold # the percentile is determined based on numpy viasualization I guess? no? how do you determine percentile?
    breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold) # If you want more chunks, lower the percentile cutoff

    # Then we'll see how many distances are actually above this one
    num_distances_above_theshold = len([x for x in distances if x > breakpoint_distance_threshold]) # The amount of distances above your threshold

    # Then we'll get the index of the distances that are above the threshold. This will tell us where we should split our text
    indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold] # The indices of those breakpoints on your list

    # Initialize the start index
    start_index = 0

    # Create a list to hold the grouped sentences
    chunks = []

    # Iterate through the breakpoints to slice the sentences
    for index in indices_above_thresh:
        # The end index is the current breakpoint
        end_index = index

        # Slice the sentence_dicts from the current start index to the end index
        group = sentences[start_index:end_index + 1]
        combined_text = ' '.join([d['sentence'] for d in group])
        chunks.append(combined_text)
        
        # Update the start index for the next group
        start_index = index + 1

    # The last group, if any sentences remain
    if start_index < len(sentences):
        combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
        chunks.append(combined_text)

    return combined_text

In [ ]:
distances, sentences = calculate_cosine_distances(sentences)

### Generating queries

In [ ]:
# Selena
SYSTEN_PROMPT_QUERIES = """
### Goal ###
Tre primary objective is to produce multiple queries in the Serbian language and a list of keywords in the Serbian language from the provided context. The context repesents an answer to the query and the keywords best 
describe the context. The goal is to have query-context pairs that corelate with each other and a list of keywords that would spead-up the search in the future.

### Process Overview ###
1. Carefully read and analyze the given context text.
2. Identify all relevant keywords and what the context text is about.
3. Find the queries that best represents the given context text.

### Formatting Rules ###
- Keyword value MUST be a LIST of strings with max lenght of 5 or [null] if no relevant information is provided.
- Use double quotes for strings and escape internal quotes with a backslash (\).
- Keep the queries concise and general about the context text.
- Ensure the output is a valid JSON file, parsable by Python's json.loads().
- Strictly use only the information provided in the context text. Do not add, infer, or imagine any additional details beyond what is explicitly stated.
- Remember to answer in Serbian.

### Query description###
- All queries must be WH-questions.
- All queries must begin with a capital letter and end with a questoin mark (?).

### Output Format ###
{{
 "keywords": ["The keyword that best represent the given context with max lenght of 5"],
 "short_query": "A short query that best suits the given context. It should be only a few words long and general.",
 "medium_query": "A minium lenght query that best suits the given context. It should be a lenght of min 10 words and max 18.",
 "long_query": "A long query that best suits the given context. It should be longer than 19 words and very specific to the context."
}}

### Context ###
{context}
"""

In [ ]:
# Selena
# def test_api(prompt, context, model: str = "gpt-3.5-turbo-0125"):
#     client = openai


#     # Assuming OpenAI API key is set elsewhere
#     # Initiate the streaming response
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "system", "content": prompt},
#             {"role": "user", "content": context}
#         ],
#         response_format={'type': 'json_object'},
#         temperature=0,
#         top_p=1
#     )

#     return response


### Creating dataset 

In [ ]:
# # Selena

# def make_dataset(file_path: Path):
#         returned_dict = {
#              "context": [],
#              "short_query": [],
#              "medium_query": [],
#              "long_query": [],
#              "keywords": []
#         }
#         # Open and iterate through the .jsonl file
#         with open(file_path, 'r') as file:
#             for line in file:
#                 data = json.loads(line)
#                 context = data[-1]['context']
#                 returned_data = data[1]['choices'][0]['message']['content']
#                 returned_data = json.loads(returned_data)
#                 returned_dict['context'].append(context)
#                 returned_dict['short_query'].append(returned_data['short_query'])
#                 returned_dict['medium_query'].append(returned_data['medium_query'])
#                 returned_dict['long_query'].append(returned_data['long_query'])
#                 returned_dict['keywords'].append(returned_data['keywords'])
#         return returned_dict